In this notebook, we show a simple example for doing Exact inference in Bayesian Networks using pgmpy. We will be using the Asia network (http://www.bnlearn.com/bnrepository/#asia) for this example.

### Step 1: Define the model.

# Fetch the asia model from the bnlearn repository

from pgmpy.utils import get_example_model

asia_model = get_example_model('asia')

In [2]:
print("Nodes: ", asia_model.nodes())
print("Edges: ", asia_model.edges())
asia_model.get_cpds()

Nodes:  ['asia', 'tub', 'smoke', 'lung', 'bronc', 'either', 'xray', 'dysp']
Edges:  [('asia', 'tub'), ('tub', 'either'), ('smoke', 'lung'), ('smoke', 'bronc'), ('lung', 'either'), ('bronc', 'dysp'), ('either', 'xray'), ('either', 'dysp')]


[<TabularCPD representing P(asia:2) at 0x23b08b3eeb0>,
 <TabularCPD representing P(bronc:2 | smoke:2) at 0x23b08b3ee80>,
 <TabularCPD representing P(dysp:2 | bronc:2, either:2) at 0x23b08b3ebb0>,
 <TabularCPD representing P(either:2 | lung:2, tub:2) at 0x23b08b3e760>,
 <TabularCPD representing P(lung:2 | smoke:2) at 0x23b08b3e220>,
 <TabularCPD representing P(smoke:2) at 0x23b08b3e370>,
 <TabularCPD representing P(tub:2 | asia:2) at 0x23b08b3e970>,
 <TabularCPD representing P(xray:2 | either:2) at 0x23b08b3eac0>]

If you would like to create a model from scratch, please refer to the Creating Bayesian Networks notebook: https://github.com/pgmpy/pgmpy/blob/dev/examples/Creating%20a%20Bayesian%20Network.ipynb

### Step 2: Initialize the inference class

Currently, pgmpy support two algorithms for inference: 1. Variable Elimination and, 2. Belief Propagation. Both of these are exact inferece algorithms. The following example uses `VariableElimination` but `BeliefPropagation` has an identifcal API, so all the methods show below would also work for `BeliefPropagation`.

In [3]:
# Initializing the VariableElimination class
"""
VariableElimination继承Inference（推理问题都要继承的基础类）
Inference类初始化时check model
并对模型类别进行判断：JunctionTree/BayesianModel...，根据不同类型，将不同的参数赋进来
"""
from pgmpy.inference import VariableElimination
asia_infer = VariableElimination(asia_model)

### Step 3: Doing Inference

In [4]:
# Computing the probability of bronc given smoke=no.
"""
VariableElimination类的query函数，可以传进的参数：（要求概率的变量，evidence,
               消元顺序（传入str时默认minFill最小缺边搜索，将采用启发式搜索，包括：  heuristic_dict = {
                "weightedminfill": WeightedMinFill,——升级版最小缺边搜索，计算的是缺边的权值和而非个数和
                "minneighbors": MinNeighbors,——即最大势搜索，最少相邻节点个数优先
                "minweight": MinWeight,——势的和
                "minfill": MinFill,——最小缺边搜索，删掉该点后，其他本来连通的点之间要增加的边数，在EliminationOrder类中
                                    MinFill函数中调用fill_in_edges获得，
                                    该函数返回combinations(self.bayesian_model.neighbors(node), 2)
                                    这里有疑问：是在有向图上进行而非端正图，对于邻居节点间已经有连线的，则多计算了
                                    问了下老师嘿嘿，由于是启发式搜索都不是精确算法，作者可能认为这样的效果也不错
            }，传入none或amrkov返回随机序列），joint：返回联合分布还是各自的）

在query函数中调用_variable_elimination函数：
1、处理输入信息（比如变量为空时...）
2、准备要用的数据：消元变量用set;working_factors（概率参数）调用_get_working_factors(evidence)，获得因子
                              把evidence给出的值代入，reduce相关的变量和因子
                            
                               elimination_ordor(消元顺序集合)调用_get_elimination_order函数获得，根据传入的消元方式，
                               调用相应的cost函数获得对每个node的score，取Score最低的放入结果序列，在图中删除，
                               循环进行直至都放入结果中。
                               
3、开始变量消元：
对于消元顺序集合（按照消元先后顺序排好的变量集合）中的每一个变量：
          1、找到与要消去变量有关的因子：factors=因子中的变量与已消元变量相交的因子，用了set().intersection()
          2、计算要消去变量的乘积：phi=factor_product(*factor)调用factor_product函数，函数中返回传入参数的乘积，用了reduce(lambda)
          3、变量边缘化/对干扰变量积分/变量消除marginal：通过累加一个变量的可能值以判定另一个变量的边缘分布
                      getattr()计算phi的属性marginalize，如果不存在触发异常，marginalize的参数传入当前正在消元变量和inplace=false
                      在DescreteFactor中inplace=false返回一个新的DescreteFactor实例，=true返回none
                      删除phi中变量var，更新cardinality和del_state_names，
                      更新计算phi.values：np.sum(phi.values,axis=tuple(var_indexes))在axis轴上展开进行求和
                             values对应变量不同取值时各自出现的次数比如A=0,B=1,C=0,出现次数=1
          4、从working_factors中删除与var相关的，加入更新好的没有var的phi
          5、将该变量var加入已消元变量
          
4、消元完毕，计算h(A)，计算后验概率，返回
          1、用final_distribution存储已消元变量之外的变量相关的因子和对应的值
          2、如果参数中joint为True（默认），返回联合分布：final_distribution的乘积的normalize结果，normalize:使概率和为1（因为原来是次数）
          3、否则，返回各自的分布：对每个要查询的变量，对除它之外的变量进行边缘化，然后normalize

        
"""
q = asia_infer.query(variables=['bronc'], evidence={'smoke': 'no'})
print(q)

# Computing the joint probability of bronc and asia given smoke=yes
q = asia_infer.query(variables=['bronc', 'asia'], evidence={'smoke': 'yes'})
print(q)

# Computing the probabilities (not joint) of bronc and asia given smoke=no
q = asia_infer.query(variables=['bronc', 'asia'], evidence={'smoke': 'no'}, joint=False)
for factor in q.values():
    print(factor)

Eliminating: xray: 100%|██████████| 5/5 [00:00<00:00, 392.84it/s]

+------------+--------------+
| bronc      |   phi(bronc) |
+============+==============+
| bronc(yes) |       0.3000 |
+------------+--------------+
| bronc(no)  |       0.7000 |
+------------+--------------+
+------------+-----------+-------------------+
| bronc      | asia      |   phi(bronc,asia) |
+============+===========+===================+
| bronc(yes) | asia(yes) |            0.0060 |
+------------+-----------+-------------------+
| bronc(yes) | asia(no)  |            0.5940 |
+------------+-----------+-------------------+
| bronc(no)  | asia(yes) |            0.0040 |
+------------+-----------+-------------------+
| bronc(no)  | asia(no)  |            0.3960 |
+------------+-----------+-------------------+
+------------+--------------+
| bronc      |   phi(bronc) |
+============+==============+
| bronc(yes) |       0.3000 |
+------------+--------------+
| bronc(no)  |       0.7000 |
+------------+--------------+
+-----------+-------------+
| asia      |   phi(asia) |
+======

In [5]:
# Computing the MAP of bronc given smoke=no.
"""
知识点：已知贝叶斯网某些变量的取值，计算另外一些变量的后验概率分布（即后验概率问题，就是变量消元算法做的事情）。（证据变量，查询变量）
       MAP问题：使得一些变量的后验概率最大的状态组合，这些变量称为假设变量H，H的一个状态组合称为一个假设h。
       后验概率最大的假设h*为最大后验假设。最大后验假设问题为MAP问题。

整体思路：调用_variable_elimination函数，在最后结果中找到概率最大的一行，输出变量取值
1、调用variable__elimination函数，结果存入final_distribution中
2、使用argmax(final_distribution)，获得最大值的索引
3、获得对应变量状态组合即最大后验假设
"""
q = asia_infer.map_query(variables=['bronc'], evidence={'smoke': 'no'})
print(q)

# Computing the MAP of bronc and asia given smoke=yes
q = asia_infer.map_query(variables=['bronc', 'asia'], evidence={'smoke': 'yes'})
print(q)
"""
对于下面提出的推理太慢的解决方案的翻译汇总：
1、变量状态太多，适当合并
2、消元顺序可选
3、自定义消元顺序，输入一个list
4、还是太慢，使用approximate推理而非精确推理
"""

Eliminating: xray: 100%|██████████| 5/5 [00:00<00:00, 558.72it/s]

{'bronc': 'no'}
{'bronc': 'yes', 'asia': 'no'}


### Step 4: Troubleshooting for slow inference

In the case of large models, or models in which variables have a lot of states, inference can be quite slow. Some of the ways to deal with it are:

1. Reduce the number of states of variables by combining states together.
2. Try a different elimination order by specifying `elimination_order` argument. Possible options are: MinFill, MinNeighbors, MinWeight, WeightedMinFill. 
3. Try a custom elimination order: The implemented heuristics for computing the elimination order might not be efficient in every case. If you can think of a more efficient order, you can also pass it as a list to the `elimination_order` argument.
4. If it is still too slow, try using approximate inference using sampling algorithms.